In [8]:
import pandas as pd
import numpy as np
from pprint import pprint
from datetime import date, timedelta, datetime
import re
from dateutil import parser


In [9]:
df = pd.read_csv('../agg_data.csv')

In [10]:
manual_corrections = {
    'Nov 2014 - Apr 2015': 'November 2014-April 2015',
    'January-April 2018': 'January 2018-April 2018',
    'October 2015, 2016': 'October 2015-October 2016',
    'February - March 2017': 'February 2017-March 2017',
    '43659': 'July 2013',  # Looked up in paper
    'July-October 2010, January-Februrary 2011': 'July 2010-February 2011',
    'January - March 2017': 'January 2017-March 2017',
    '43478': 'January 2013',  # Looked up in paper
    '8-Jul': 'October 2016',   # Looked up in paper
    'July - December 2016': 'July 2016-December 2016',
    'March-May 2014': 'March 2014-May 2014',
    '14-Nov': 'November 2014',  # Looked up in paper
    'June-July 2013': 'June 2013-July 2013',
    'Nov-Dec 2014, 2015': 'November 2014-December 2015',
    '13-Jul': 'September 2013-January 2014',   # Looked up in paper
    'Feb. -April 2007': 'February 2007-April 2007',
    '2009-2013?': '2009-2013',
    '18-Jan': 'September 2017',   # Locked up in paper
    'summer and autumn 2010': 'Summer 2010-Fall 2010',
    'August-September 2005': 'August 2005-September 2005',
    'July 1970 - August 1971': 'July 1970-August 1971',
    'Sep-10': "September 2010",
    'May-11': 'May 2011',
    'Jul-13': 'July 2013',
    'Jan-18': 'January 2018',
    'Mar-17': 'March 2017'
}

# Replace values in the original column using the manual_corrections dictionary
df['Year of collection'] = df['Year of collection'].replace(manual_corrections)

In [11]:
def parse_date(text):
    # Handle NaN values directly
    if pd.isna(text):
        return np.nan
    
    try:
        # Year only, e.g., '2014'
        if re.match(r'^\d{4}$', text):
            return (pd.to_datetime(text + '-01-01'), pd.to_datetime(text + '-12-31'))
        
        # Season and year, e.g., 'Summer 2016'
        elif re.match(r'^(Spring|Summer|Fall|Winter) \d{4}$', text, re.IGNORECASE):
            season, year = text.split()
            if season.lower() == 'spring':
                start_month = '03'
                end_month = '05'
            elif season.lower() == 'summer':
                start_month = '06'
                end_month = '08'
            elif season.lower() == 'fall':
                start_month = '09'
                end_month = '11'
            elif season.lower() == 'winter':
                start_month = '12'
                end_month = '02'
            return (pd.to_datetime(f"{year}-{start_month}-01"), pd.to_datetime(f"{year}-{end_month}-30"))
        
        # Year range, e.g., '2012-2022'
        elif re.match(r'^\d{4}-\d{4}$', text):
            start_year, end_year = text.split('-')
            return (pd.to_datetime(start_year + '-01-01'), pd.to_datetime(end_year + '-12-31'))
        
        # Season range, e.g., 'Spring 2012-Fall 2015'
        elif re.match(r'^(Spring|Summer|Fall|Winter) \d{4}-(Spring|Summer|Fall|Winter) \d{4}$', text, re.IGNORECASE):
            start_season, start_year, end_season, end_year = re.split(r'[- ]', text)
            season_map_start = {'spring': '03', 'summer': '06', 'fall': '09', 'winter': '12'}
            season_map_end = {'spring': '05', 'summer': '08', 'fall': '11', 'winter': '02'}
            start_month = season_map_start[start_season.lower()]
            end_month = season_map_end[end_season.lower()]
            return (pd.to_datetime(f"{start_year}-{start_month}-01"), pd.to_datetime(f"{end_year}-{end_month}-30"))
        
        # Month range, e.g., 'January 2012-March 2013'
        elif re.match(r'^[A-Za-z]+ \d{4}-[A-Za-z]+ \d{4}$', text):
            start_month_year, end_month_year = text.split('-')
            start_date = pd.to_datetime(start_month_year)
            end_date = pd.to_datetime(end_month_year)
            return (start_date, end_date)
        
        # Month and year, e.g., 'November 2011'
        elif re.match(r'^[A-Za-z]+ \d{4}$', text):
            return (pd.to_datetime(text), pd.to_datetime(text) + pd.offsets.MonthEnd(0))
        
        # Full date (if in ambiguous format, can adjust format here)
        else:
            return (pd.to_datetime(text, errors='coerce'), pd.to_datetime(text, errors='coerce') + pd.offsets.MonthEnd(0))
        
    except Exception as e:
        print(f"Error parsing date '{text}': {e}")
        return np.nan

# Apply the function to create a new column
df['parsed_date'] = df['Year of collection'].apply(parse_date)




In [12]:
missing_rows = df[df['parsed_date'].isna()]

missing_rows['Year of collection'].unique()

array([nan], dtype=object)

In [13]:
def risk_category(col, no_risk_threshold, low_risk_threshold):
        if col <= no_risk_threshold:
            return 0  # No risk
        elif col <= low_risk_threshold:
            return 1  # Low risk
        else:
            return 2  # High risk

In [14]:
# Apply the function to create a new column
df['Risk_0_3'] = df['Mean num particles per indv'].apply(lambda x: risk_category(x, 0, 3))
df['Risk_0_5'] = df['Mean num particles per indv'].apply(lambda x: risk_category(x, 0, 5))
df['Risk_0_10'] = df['Mean num particles per indv'].apply(lambda x: risk_category(x, 0, 10))
df['Risk_0pt5_5'] = df['Mean num particles per indv'].apply(lambda x: risk_category(x, 0.5, 5))
df['Risk_0pt5_10'] = df['Mean num particles per indv'].apply(lambda x: risk_category(x, 0.5, 10))
df['Risk_1_5'] = df['Mean num particles per indv'].apply(lambda x: risk_category(x, 1, 5))
df['Risk_1_10'] = df['Mean num particles per indv'].apply(lambda x: risk_category(x, 1, 10))
df['Risk_3_10'] = df['Mean num particles per indv'].apply(lambda x: risk_category(x, 3, 10))
df['Risk_5_10'] = df['Mean num particles per indv'].apply(lambda x: risk_category(x, 5, 10))

In [15]:
df.to_csv('../agg_data_cleaned.csv')